In [1]:
import numpy as np

In [185]:
import numpy as np

class HubbardHartreeFock:
    def __init__(self, t, U, N):
        """
        Initializes the Hubbard model parameters for Hartree-Fock calculations.

        Parameters:
        - t (float): Hopping parameter.
        - U (float): On-site interaction strength.
        - N (int): Number of k-points in each direction of the 2D lattice.
        """
        self.t = t  # Hopping parameter
        self.U = U  # On-site interaction strength
        self.N = N  # Number of k-points in each direction
        self.k_space = self.generate_k_space()  # Generating the k-space
        self.H0 = self.generate_noninteracting()
        # Placeholder for H0 initialization
        # Placeholder for exp_val initialization

    def generate_k_space(self):
        """
        Generates the mesh grid of k points in the first Brillouin zone.

        Returns:
        - numpy.ndarray: A 2D array with shape (N^2, 2), where the first column
                         is the kx component and the second column is the ky component.
        """
        k_values = np.linspace(-np.pi, np.pi, self.N, endpoint=False)
        kx, ky = np.meshgrid(k_values, k_values)
        k_space = np.vstack([kx.flatten(), ky.flatten()]).T
        return k_space

    def generate_noninteracting(self):
        """
        Constructs the noninteracting term for the Hamiltonian.

        Returns:
        - numpy.ndarray: A 3D array with shape (2, 2, N^2), where the first two axes
                         represent the spin components, and the third axis represents
                         different k points.
        """
        # Initializing the array
        noninteracting_matrix = np.zeros((2, 2, self.N**2))

        # Calculating epsilon_k for each k point
        epsilon_k = -2 * self.t * (np.cos(self.k_space[:, 0]) + np.cos(self.k_space[:, 1]))

        # Filling in the diagonal for both spin up and down
        noninteracting_matrix[0, 0, :] = epsilon_k  # Spin up diagonal
        noninteracting_matrix[1, 1, :] = epsilon_k  # Spin down diagonal

        return noninteracting_matrix

    def generate_H_int(self, exp_val):
        """
        Constructs the interacting term after Hartree-Fock decomposition.

        Parameters:
        - exp_val (numpy.ndarray): Expected values with shape (2, N^2), where the first index
                                   represents spin up and down, and the second index represents
                                   different k points.

        Returns:
        - numpy.ndarray: A 3D array with shape (2, 2, N^2), representing the interacting term.
        """
        # Initializing the interacting term matrix
        H_int_matrix = np.zeros((2, 2, self.N**2), dtype=np.complex128)
        
        # Calculating the interacting term for both spin components
        # Note: The sum over k' is accounted for by averaging the expectation values (exp_val)
        avg_exp_val_up = np.mean(exp_val[0, :])  # Average for spin up
        avg_exp_val_down = np.mean(exp_val[1, :])  # Average for spin down
        
        # Filling the diagonal terms according to the Hartree-Fock approximation
        H_int_matrix[0, 0, :] = self.U * avg_exp_val_down  # For spin up electrons
        H_int_matrix[1, 1, :] = self.U * avg_exp_val_up  # For spin down electrons

        return H_int_matrix

    def get_occupancy(self, en, nu):
        """
        Calculates the occupancy of each state at each k point.

        Parameters:
        - en (numpy.ndarray): The energies with shape (2, N^2), where the first index is the level index
                              (spin up and spin down), and the second index is for different k points.
        - nu (float): The filling factor, determining the proportion of occupied states.

        Returns:
        - numpy.ndarray: The occupancy with shape (2, N^2), where the first index is the level index
                         (spin up and spin down), and the second index is for different k points.
        """
        # Flatten the energy array to find the universal Fermi level
        flattened_en = en.flatten()

        # Sort the flattened energy array to find the Fermi level based on filling factor
        sorted_en = np.sort(flattened_en)
        
        # Determine the index for the Fermi level based on the filling factor
        fermi_index = int(np.floor(nu * len(sorted_en)))
        
        # Fermi energy is the energy at the Fermi index
        fermi_energy = sorted_en[fermi_index-1]

        # Compute occupancy: 1 if energy <= Fermi level, 0 otherwise
        occupancy = np.where(en <= fermi_energy, 1, 0)

        return occupancy

    def compute_exp_val(self, wf, occupancy):
        """
        Computes the expected value <c_k,s^dagger c_k,s> using the wavefunction and occupancy.

        Parameters:
        - wf (numpy.ndarray): Wavefunction with shape (2, 2, N^2), where the first index is for spin (up/down),
                              the second index is the level index, and the third index is for different k points.
        - occupancy (numpy.ndarray): Occupancy of each state at each k point, with shape (2, N^2),
                                     where the first index is the level index, and the second index is for different k points.

        Returns:
        - numpy.ndarray: The expected value with shape (2, N^2), where the first index is for spin (up/down),
                         and the second index is for different k points.
        """
        # Compute the expected value using tensor contraction with np.einsum
        exp_val = np.einsum('slk,lk,slk->sk', wf, occupancy, np.conj(wf))
        return exp_val



    def generate_total(self, H_int):
        """
        Computes the total Hamiltonian as the sum of the noninteracting and interacting terms.

        Parameters:
        - H_int (numpy.ndarray): The interacting term with shape (2, 2, N^2), where the first two indices
                                 are for spin up and spin down, and the third axis is for different k points.

        Returns:
        - numpy.ndarray: The total Hamiltonian with the same shape (2, 2, N^2).
        """
        # Summing the noninteracting term H0 and the interacting term H_int
        H_total = self.H0 + H_int
        return H_total

    def diagonalize(self, H_total):
        """
        Diagonalizes the total Hamiltonian for each k point, sorts the eigenvalues and eigenvectors.

        Parameters:
        - H_total (numpy.ndarray): The total Hamiltonian with shape (2, 2, N^2).

        Returns:
        - numpy.ndarray: Eigenvectors (wavefunctions) with shape (2, 2, N^2).
        - numpy.ndarray: Eigenvalues (energies) with shape (2, N^2).
        """
        N2 = self.N**2
        wf = np.zeros((2, 2, N2), dtype=np.complex128)  # Wavefunctions (eigenvectors)
        en = np.zeros((2, N2))  # Eigenvalues (energies)

        # Ensure H_total is Hermitian by symmetrizing it
        H_total_sym = (H_total + np.conjugate(np.swapaxes(H_total, 0, 1))) / 2

        # Loop over each k point
        for i in range(N2):
            # Diagonalize the 2x2 Hamiltonian for this k point
            vals, vecs = np.linalg.eigh(H_total_sym[:, :, i])

            # Sort eigenvalues and eigenvectors
            sort_index = np.argsort(vals)
            vals_sorted = vals[sort_index]
            vecs_sorted = vecs[:, sort_index]

            # Store the sorted eigenvalues and eigenvectors
            en[:, i] = vals_sorted
            wf[:, :, i] = vecs_sorted

        return wf, en

    def get_energy(self, exp_val):
        """
        Computes the wavefunction and eigenvalues from the expected value.

        Parameters:
        - exp_val (numpy.ndarray): Expected values with shape (2, N^2), for spin up and down, and different k points.

        Returns:
        - numpy.ndarray: Wavefunctions with shape (2, 2, N^2).
        - numpy.ndarray: Eigenvalues with shape (2, N^2).
        """
        # Step 1: Compute the interacting term H_int from exp_val
        H_int = self.generate_H_int(exp_val)

        # Step 2: Compute the total Hamiltonian H_total from H_int and H0
        H_total = self.generate_total(H_int)

        # Step 3: Diagonalize H_total to get wavefunctions and energies
        wf, en = self.diagonalize(H_total)

        return wf, en

    def get_exp_val(self, wf, en, nu):
        """
        Computes the expected values from the wavefunction, eigenenergies, and filling factor.

        Parameters:
        - wf (numpy.ndarray): Wavefunctions with shape (2, 2, N^2).
        - en (numpy.ndarray): Eigenenergies with shape (2, N^2).
        - nu (float): Filling factor.

        Returns:
        - numpy.ndarray: Expected values with shape (2, N^2).
        """
        # Compute the occupancy based on energies and filling factor
        occ = self.get_occupancy(en, nu)

        # Compute the expected value from wavefunction and occupancy
        exp_val = self.compute_exp_val(wf, occ)

        return exp_val

    def solve(self, exp_val_0, nu):
        """
        Self-consistently solves the Hubbard model using Hartree-Fock approximation.

        Parameters:
        - exp_val_0 (numpy.ndarray): Initial ansatz for the expected value, shape (2, N^2).
        - nu (float): Filling factor.

        Returns:
        - numpy.ndarray: Final wavefunction, shape (2, 2, N^2).
        - numpy.ndarray: Final eigenvalues (energies), shape (2, N^2).
        - numpy.ndarray: Final expected values, shape (2, N^2).
        """
        # Initialize the expected value with the initial ansatz
        exp_val = exp_val_0

        for iteration in range(100):  # Loop for a fixed number of iterations
            # Step 1: Get energy and wavefunction from the current expected value
            wf, en = self.get_energy(exp_val)

            # Step 2: Update the expected value from the new wavefunction, energies, and filling factor
            new_exp_val = self.get_exp_val(wf, en, nu)

            # Check for convergence (optional improvement could involve setting a tolerance)
            if np.allclose(new_exp_val, exp_val, atol=1e-10):
                print(f"Convergence reached at iteration {iteration}")
                break

            # Update the expected value for the next iteration
            exp_val = new_exp_val

        # Return the final wavefunction, energies, and expected values
        return wf, en, exp_val



# Check Kinetic

## Point Group

### Rotation

In [8]:
from copy import copy

In [75]:
rot_mat=lambda theta: np.array([[np.cos(theta),-np.sin(theta)],[np.sin(theta),np.cos(theta)]])

In [76]:
def rotate_2D(k_space,theta):
    """k_space: k-mesh, np.ndarray(N,2)
    theta: rotate angle (counterclockwise)"""
    assert k_space.shape[-1]==2, f"Only 2D lattice is supported, the current dimension of lattice is {k_space.shape[-1]}"
    
    return k_space@rot_mat(theta).T

In [109]:
def check_rotation(model,kind):
    """R_θ H(k) R_θ.T=H(R_θ(k))=H(k)
    """
    theta={'C'+str(n):np.pi*2/n for n in [2,3,4,6]}
    assert kind in theta, f'Symmetry should be among f{angle.keys()}'
    ek=model.generate_noninteracting()
    model_1=copy(model)
    model_1.k_space=rotate_2D(hb_1.k_space,theta=theta[kind])
    
    ek_1=model_1.generate_noninteracting()
    return np.allclose(ek,ek_1)

In [110]:
hb=HubbardHartreeFock(t=1,U=0,N=100)

In [111]:
check_rotation(hb,kind='C2')

True

In [112]:
check_rotation(hb,kind='C4')

True

In [113]:
check_rotation(hb,kind='C3')

False

### mirror

In [82]:
def mirror_2D(k_space,theta):
    """k_space: k-mesh, np.ndarray(N,2)
    theta: angle of mirror axis
    mirror= R(-θ) @ M @ R(θ)
    """
    assert k_space.shape[-1]==2, f"Only 2D lattice is supported, the current dimension of lattice is {k_space.shape[-1]}"
    rot_mat(-theta)
    inv_mat=np.diag([1,-1])
    mirror=rot_mat(theta)@inv_mat@rot_mat(-theta)
    return k_space@mirror.T

In [83]:
mirror_2D(np.array([[1,0]]),theta=np.pi/4)

array([[2.22044605e-16, 1.00000000e+00]])

In [87]:
ek.shape

(2, 2, 10000)

In [120]:
def check_mirror(model,theta):
    """
    y=tan(θ)x
    M_θ= R(θ) M_y R(-θ)
    M_θ H_{σ,σ'}(k) M_θ^(-1) = \\tilde{R}_θ H_{σ,σ'}(k) \\tilde{R}_θ^(-1), where \\tilde{R}_θ = R(θ) (-iσ_y) R(-θ)
    """
    ek=model.generate_noninteracting()
    model_1=copy(model)
    model_1.k_space=mirror_2D(hb_1.k_space,theta=theta)
    i_pauli_y=np.array([[0,1],[-1,0]])
    R_theta=rot_mat(theta)@(-i_pauli_y)@rot_mat(-theta)
    ek_1=np.einsum(R_theta,[0,1],model_1.generate_noninteracting(),[1,2,4],R_theta.T,[2,3],[0,3,4])
    return np.allclose(ek,ek_1)

In [121]:
check_mirror(hb,theta=0)

True

In [122]:
check_mirror(hb,theta=np.pi/4)

True

In [123]:
check_mirror(hb,theta=np.pi/2)

True

In [124]:
check_mirror(hb,theta=np.pi/4*3)

True

In [125]:
check_mirror(hb,theta=np.pi/8)

False

## Time-reversal symmetry

In [72]:
ek.shape

(2, 2, 10000)

In [ ]:
def TRS_spinless(model):
    """T=K, T^=1
    T H(k) T^{-1}=H(-k)^*
    """
    ek=model.generate_noninteracting()
    model_1=copy(model)
    model_1.k_space=rotate_2D(hb_1.k_space,theta=np.pi) #k->-k
    ek_1=model_1.generate_noninteracting()
    return np.allclose(ek,ek_1)


In [134]:
def check_TRS_spinful(model):
    """T=i\sigma_y K, T^2=-1
    T H_{σ,σ'}(k) T^(-1) = (iσ_y) * H_{σ,σ'}(-k).conj * (-iσ_y) = H_{σ,σ'}(k)
    """
    ek=model.generate_noninteracting()
    model_1=copy(model)
    model_1.k_space=-hb_1.k_space #k->-k
    i_pauli_y=np.array([[0,1],[-1,0]])
    ek_1=np.einsum((i_pauli_y),[0,1],model_1.generate_noninteracting().conj(),[1,2,4],(-i_pauli_y),[2,3],[0,3,4])
    return np.allclose(ek,ek_1)

In [135]:
check_TRS_spinful(hb,)

True

## Chiral

In [ ]:
# TBD

# def check_Chiral_spinful(model):
#     """T=i\sigma_y K, T^2=-1
#     T H_{σ,σ'}(k) T^(-1) = (iσ_y) * H_{σ,σ'}(-k).conj * (-iσ_y) = H_{σ,σ'}(k)
#     """
#     ek=model.generate_noninteracting()
#     model_1=copy(model)
#     model_1.k_space=-hb_1.k_space #k->-k
#     i_pauli_y=np.array([[0,1],[-1,0]])
#     ek_1=np.einsum((i_pauli_y),[0,1],model_1.generate_noninteracting().conj(),[1,2,4],(-i_pauli_y),[2,3],[0,3,4])
#     return np.allclose(ek,ek_1)

# Interaction

In [230]:
def check_interaction_gap(model):
    model_1=copy(model)
    exp_val=np.random.random((hb.H0.shape[0],hb.H0.shape[-1],))*0.5
    wf,en,exp_val=hb.solve(exp_val,0.5)
    en_sort=np.sort(en.flatten())
    gap=en_sort[en_sort.shape[0]//2]-en_sort[en_sort.shape[0]//2-1]
    return np.allclose(gap,model_1.U)

In [231]:
hb=HubbardHartreeFock(t=0,U=2,N=100)
check_interaction_gap(hb)

Convergence reached at iteration 1


True